<h1 align="center">LAB1_S3. Static word-embeddings for text representation</h1>

<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

In [1]:
#Installing Gensim library
!pip install -U gensim
!pip install -U nltk
!pip install -U fasttext-wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 11.7 MB/s eta 0:00:00


## Some libraries

In [2]:
import fasttext.util
import gensim.downloader as api
from gensim.models.keyedvectors import KeyedVectors
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity

nltk.download("punkt_tab")#nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Load both corpora

In [3]:
path_english = "EXIST2024_EN_examples.csv"
path_spanish = "EXIST2024_ES_examples.csv"

#df = {
#    "english": pd.read_csv(path_english, sep="\t"),
#    "spanish": pd.read_csv(path_spanish, sep="\t")
#}

# OR USING Google colab
from google.colab import drive
drive.mount('/content/drive')
df = {
    "english": pd.read_csv("/content/drive/MyDrive/LNR/EXIST2024_EN_examples.csv", sep="\t"),
    "spanish": pd.read_csv("/content/drive/MyDrive/LNR/EXIST2024_ES_examples.csv", sep="\t")
}


ValueError: mount failed

## Preprocess and tokenize the corpora:

- remove URLs
- remove hashtags
- remove users
- lowercase
- tokenize (using _word_tokenize_)
- remove stopwords (using nltk stopwords)

Note: tokenization and stopword removal are language-dependent.

In [ ]:
# Info: nltk.tokenize.word_tokenize(text, language='english', preserve_line=False)

web_re = re.compile(r"https?:\/\/[^\s]+", re.U)
user_re = re.compile(r"(@\w+\-?(?:\w+)?)", re.U)
hashtag_re = re.compile(r"(#\w+\-?(?:\w+)?)", re.U)

stopw = {
    "english": nltk.corpus.stopwords.words("english"),
    "spanish": nltk.corpus.stopwords.words("spanish")
}

def preprocess(text):
    text = web_re.sub("", text)  # Remove URLs
    text = user_re.sub("", text)  # Remove user mentions
    text = hashtag_re.sub("", text)  # Remove hashtags
    text = text.lower()  # Convert to lowercase
    return text

def tokenize(text_list, lang="english"):
    # COMPLETE
    token_list = []
    for text in text_list:
        cleaned_text = preprocess(text)
        tokens = word_tokenize(cleaned_text, language=lang)
        filtered_tokens = [word for word in tokens if word not in stopw[lang]]
        token_list.append(filtered_tokens)
    return token_list

tokenized_text = {
    "english": tokenize(df["english"]["text"], "english"),
    "spanish": tokenize(df["spanish"]["text"], "spanish")
}

t = ["Hola, mi nombre es Antonio, ¿todo bien? https://www.upv.es @paquita", "Hi! my name is Peter"]
print(t)
print(tokenize(t, "spanish"))
print(tokenize(t, "english"))

['Hola, mi nombre es Antonio, ¿todo bien? https://www.upv.es @paquita', 'Hi! my name is Peter']
[['hola', ',', 'nombre', 'antonio', ',', '¿todo', 'bien', '?'], ['hi', '!', 'my', 'name', 'is', 'peter']]
[['hola', ',', 'mi', 'nombre', 'es', 'antonio', ',', '¿todo', 'bien', '?'], ['hi', '!', 'name', 'peter']]


## Text representation using static embeddings

ENGLISH

- word2vec-google-news-300 (using Gemini)
- fasttext-wiki-news-subwords-300 (using Gemini)
- glove-wiki-gigaword-300 (using Gemini)

SPANISH
- Fasttext (https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz) (using Gemini)

### Load the models

In [ ]:
#from google.colab import drive
#import gensim.downloader as api

# Montar Google Drive
#drive.mount('/content/drive')

# Definir rutas para guardar los modelos
#base_path = "/content/drive/MyDrive/gensim_models/"
#word2vec_path = base_path + "word2vec-google-news-300.model"
#fasttext_path = base_path + "fasttext-wiki-news-subwords-300.model"
#glove_path = base_path + "glove-wiki-gigaword-300.model"

# Descargar y guardar modelos
#word2vec = api.load("word2vec-google-news-300")
#word2vec.save(word2vec_path)

#fasttext_en = api.load("fasttext-wiki-news-subwords-300")
#fasttext_en.save(fasttext_path)

#glove = api.load("glove-wiki-gigaword-300")
#glove.save(glove_path)

#print("Modelos guardados en Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Modelos guardados en Google Drive.


In [ ]:
from google.colab import drive
from gensim.models import KeyedVectors

# Montar Google Drive
drive.mount('/content/drive')

# Definir rutas de los modelos
base_path = "/content/drive/MyDrive/gensim_models/"
word2vec_path = base_path + "word2vec-google-news-300.model"
fasttext_path = base_path + "fasttext-wiki-news-subwords-300.model"
glove_path = base_path + "glove-wiki-gigaword-300.model"

# Cargar modelos desde el almacenamiento local
word2vec = KeyedVectors.load(word2vec_path)
fasttext_en = KeyedVectors.load(fasttext_path)
glove = KeyedVectors.load(glove_path)

es_model = KeyedVectors.load_word2vec_format("/content/cc.es.300.bin.gz", binary=True)

print("Modelos cargados desde Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/cc.es.300.bin.gz'

### Compute static word-embeddings representation of the tweets

In [ ]:
def gensim_sentence_rep(tokens, model):
    """
    Compute the word-embedding representation of a list of tokens by averaging
        the representations of each word
    """
    avg_vec = np.zeros(model.vector_size)
    total_w = 0

    for word in tokens:
        if word in model:
            avg_vec += model[word]
            total_w += 1

    if total_w == 0:
        return avg_vec  # Evita la división por cero
    return avg_vec / total_w

# Apply "gensim_sentence_rep"
# COMPLETE
sentence_vectors = {
    "word2vec": [gensim_sentence_rep(tokens, word2vec) for tokens in tokenized_text["english"]],
    "fasttext": [gensim_sentence_rep(tokens, fasttext_en) for tokens in tokenized_text["english"]],
    "glove": [gensim_sentence_rep(tokens, glove) for tokens in tokenized_text["english"]]
}

# Ejemplo de salida
print(sentence_vectors["word2vec"][0])


[ 0.03984649  0.02089928 -0.03820169  0.12103482 -0.03136365  0.04779
  0.00409567 -0.04767267  0.07529423  0.05294195  0.01294156 -0.09585624
 -0.05405978  0.01406966 -0.0938868   0.04490925  0.03281166  0.12748166
  0.08025478 -0.12434545 -0.00841233  0.0613135  -0.0060046  -0.00787985
  0.05632493 -0.04291102 -0.08166083  0.06243212  0.09708378 -0.04894967
  0.03260119 -0.00482809 -0.02081483 -0.00529954  0.03326291  0.05588926
  0.05469776 -0.01033388  0.06848986  0.10389762  0.08990268 -0.06543942
  0.13988416  0.00971301 -0.02455087 -0.00292337 -0.06240529 -0.04385902
 -0.02660817  0.02828138 -0.07819656  0.06540759  0.020592   -0.01371818
  0.03399448  0.00438822 -0.02529065 -0.07915207  0.02505388 -0.1243102
  0.05248392  0.04191945 -0.09331881 -0.02962283  0.06362178  0.01116312
 -0.05182135  0.02382213 -0.03370824  0.04812833  0.03661162  0.01473789
  0.07373902  0.01100317 -0.19417914 -0.03608651  0.0338219   0.07401565
  0.04823672  0.06193385 -0.00915212 -0.03922666  0.022

## Compute cosine similarities

In [ ]:
def compute_cosine_similarities(vectors):
    return cosine_similarity(vectors)

cosine_similarities = {
    "word2vec": compute_cosine_similarities(sentence_vectors["word2vec"]),
    "fasttext": compute_cosine_similarities(sentence_vectors["fasttext"]),
    "glove": compute_cosine_similarities(sentence_vectors["glove"])
}

def find_most_similar_optimized(sim_matrix, df, label_col, label):
    df_filtered = df[df[label_col] == label]
    indices = df_filtered.index.tolist()

    if len(indices) < 2:
        return (None, None), -1  # Si hay menos de dos elementos, no se puede calcular la similitud

    max_sim = -1
    best_pair = (None, None)

    for i in range(len(indices)):
        for j in range(i + 1, len(indices)):
            sim = sim_matrix[indices[i], indices[j]]
            if sim > max_sim:
                max_sim = sim
                best_pair = (df_filtered.iloc[i]['id'], df_filtered.iloc[j]['id'])

    return best_pair, max_sim

## Show results

In [ ]:
for name, sim_matrix in cosine_similarities.items():
    print(f"======================\n{name} \n----------------------")
    for label in ['NO', 'YES']:
        best_pair, similarity = find_most_similar_optimized(sim_matrix, df["english"], "label", label)
        if best_pair[0] is not None:
            print(f"Label: {label} \nTweets IDs: {best_pair} \nSimilarity: {similarity:.4f}")
            print(f"Tweets: \n \t1: {df['english'].loc[df['english']['id'] == best_pair[0], 'text'].values[0]} \n \t2: {df['english'].loc[df['english']['id'] == best_pair[1], 'text'].values[0]}")
        if label == "NO":
            print("----------------------")

word2vec 
----------------------
Label: NO 
Tweets IDs: (201173, 201177) 
Similarity: 0.9444
Tweets: 
 	1: @BLEEDTHISWAY replay free woman breebylon &gt;&gt;&gt; Flop this way 
 	2: replay&gt;alice&gt;babylon&gt;free woman https://t.co/WCEqeUxdtC
----------------------
Label: YES 
Tweets IDs: (201621, 201637) 
Similarity: 0.9614
Tweets: 
 	1: @WeaponizedRage Aerosmith in 1987: "Dude looks like a lady" 
 	2: Dude does not look like a lady! https://t.co/C62JmKSzy0
fasttext 
----------------------
Label: NO 
Tweets IDs: (201173, 201177) 
Similarity: 0.9898
Tweets: 
 	1: @BLEEDTHISWAY replay free woman breebylon &gt;&gt;&gt; Flop this way 
 	2: replay&gt;alice&gt;babylon&gt;free woman https://t.co/WCEqeUxdtC
----------------------
Label: YES 
Tweets IDs: (201235, 201978) 
Similarity: 0.9716
Tweets: 
 	1: in the living room, all strocking to porn. all4guys+host had fucked me. 1st, straight,had cum in cumhole. I was so surprised&amp; pleased.even if short &amp;light, was so hot be gangbanged